## Matching the python and tensorflow version of this notebook to the car due to the error we had while loading the trained model

In [ ]:
##Running this will force Kaggle to use Python 3.10.10, just in case you didn’t have a way

!mamba create -n py310 -y

!source /opt/conda/bin/activate py310 && mamba install python=3.10 jupyter mamba -y

 

!sudo rm /opt/conda/bin/python3

!sudo ln -sf /opt/conda/envs/py310/bin/python3 /opt/conda/bin/python3

!sudo rm /opt/conda/bin/python3.7

!sudo ln -sf /opt/conda/envs/py310/bin/python3 /opt/conda/bin/python3.7

!sudo rm /opt/conda/bin/python

!sudo ln -sf /opt/conda/envs/py310/bin/python3 /opt/conda/bin/python

# Importing libraries

In [6]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from pathlib import Path
from glob import glob
import tensorflow as tf
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import *
from keras.models import * 
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.applications import MobileNet
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

# Reading images and importing dataframes

In [7]:
def preprocessing(img_dir,csv_dir):
    image_dir = Path(img_dir)
    filepaths = pd.DataFrame(list(image_dir.glob(r'**/*.png')), columns=['Filepath']).astype(str)
    li = ['/3141.png','/3999.png','/4895.png','/8285.png','/10171.png']
    for i in range(5):
        filepaths.drop(filepaths[filepaths['Filepath']==img_dir+li[i]].index, inplace=True)
    filepaths['image_name'] = filepaths['Filepath'].str.extract('\d+.*?(\d+)')
    filepaths['image_name'] = filepaths['image_name'].astype(np.int64)
    filepaths.sort_values(by=['image_name'],inplace=True)
    df = pd.read_csv(csv_dir)
    new_df = pd.concat([filepaths.reset_index(drop=True),df[['image_id','angle','speed']].reset_index(drop=True)],axis=1)
    img_id = new_df['image_id'].to_numpy()
    return new_df, img_id

In [8]:
train_dir = '/kaggle/input/machine-learning-in-science-ii-2023/training_data/training_data'
train_csv_dir = '/kaggle/input/machine-learning-in-science-ii-2023/training_norm.csv'
train_df, train_img_id = preprocessing(train_dir,train_csv_dir)

test_dir = '/kaggle/input/machine-learning-in-science-ii-2023/test_data'
test_csv_dir = '/kaggle/input/machine-learning-in-science-ii-2023/sampleSubmission.csv'
test_df, test_img_id = preprocessing(test_dir,test_csv_dir)

### Tried importing more dataset we recorded to train the model again but couldnt due to the errors we were having while loading the model

In [10]:
image_dir = Path('/kaggle/input/new-dataset')
new_dataset = pd.DataFrame(list(image_dir.glob(r'**/*.png')), columns=['Filepath']).astype(str)
new_dataset['image_name'] = np.nan
new_dataset['image_id'] = np.nan
new_dataset['speed'] = np.nan
new_dataset['angle'] = np.nan
for i in range(len(new_dataset)):
    new_dataset['image_name'].iloc[i] = new_dataset['Filepath'].iloc[i].split('_')[0]
    new_dataset['image_id'] = new_dataset['image_name'].astype(int)
    new_dataset['speed'].iloc[i] = new_dataset['Filepath'].iloc[i].split('_')[1]
    new_dataset['angle'].iloc[i] = new_dataset['Filepath'].iloc[i].split('_')[2].split('.')[0]
    
print(new_dataset.head())


final_df = pd.concat([train_df,new_dataset], ignore_index=True)
final_df.to_csv('final_df.csv')
print(final_df.head())
print(final_df.shape)

ValueError: invalid literal for int() with base 10: '/kaggle/input/new-dataset/1683119580908'

## Creating generators to rescale the images in the range of 0-1 and splitting dataset into train, validation and test

In [11]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.25
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [12]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col=['angle','speed'],
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=160,
    shuffle=True,
    seed=42,
    subset='training'
)


val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col=['angle','speed'],
    target_size=(224,224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=160,
    shuffle=True,
    seed=42,
    subset='validation'
)
    
test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col=['angle','speed'],
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='raw',
    batch_size=160,
    shuffle=False
)

Found 10345 validated image filenames.
Found 3448 validated image filenames.
Found 1020 validated image filenames.


# Self made CNN model

In [20]:
model = Sequential()
model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

model.add(Conv2D(256,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
#model.add(Dropout(0.25))

model.add(Flatten())
#model.add(Dense(4096,activation='relu'))
#model.add(Dropout(0.4))
model.add(Dense(2048,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(512,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2,activation='linear'))

#from keras import optimizers
#optm = optimizers.Adam(learning_rate=0.001)
model.compile(loss='mean_squared_error',optimizer='adam')
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_415 (Conv2D)         (None, 222, 222, 32)      896       
                                                                 
 conv2d_416 (Conv2D)         (None, 220, 220, 64)      18496     
                                                                 
 max_pooling2d_11 (MaxPoolin  (None, 110, 110, 64)     0         
 g2D)                                                            
                                                                 
 conv2d_417 (Conv2D)         (None, 108, 108, 128)     73856     
                                                                 
 max_pooling2d_12 (MaxPoolin  (None, 54, 54, 128)      0         
 g2D)                                                            
                                                                 
 conv2d_418 (Conv2D)         (None, 52, 52, 256)      

# InceptionResnetV2 (used for kaggle)

In [21]:
model = Sequential()

pretrained_model = tf.keras.applications.InceptionResNetV2(include_top=False,weights="imagenet",input_shape=(224, 224, 3))

for layer in pretrained_model.layers:
        layer.trainable = False


#tried with different values and taken the best architecture and layers according to their evaluation measures
for layer in pretrained_model.layers[745:]:
    layer.trainable = True

model.add(pretrained_model)

model.add(GlobalAveragePooling2D())
model.add(Dense(1024, activation ='relu',kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6)))
model.add(Dropout(0.2))
model.add(Dense(256, activation ='relu',kernel_regularizer=l2(1e-7), bias_regularizer=l2(1e-7)))
model.add(Dropout(0.2))
model.add(Dense(2))

model.summary()
model.compile(optimizer = 'adam',loss='mse')

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_resnet_v2 (Functi  (None, 5, 5, 1536)       54336736  
 onal)                                                           
                                                                 
 global_average_pooling2d_5   (None, 1536)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_20 (Dense)            (None, 1024)              1573888   
                                                                 
 dropout_13 (Dropout)        (None, 1024)              0         
                                                                 
 dense_21 (Dense)            (None, 256)               262400    
                                                                 
 dropout_14 (Dropout)        (None, 256)              

# Nvidia model (It is the lightest model in any model we experimented with and had fairly decent loss. We came to know by our peers that the car (car4) we tried to run this on had some issues unfortunately, else we believe this would have a lesser inference time and faster car run) [450k parameters]

In [13]:
from tensorflow.keras.optimizers import Adam
def nvidia_model():
    model = Sequential(name='Nvidia_Model')
    
    # elu=Expenential Linear Unit, similar to leaky Relu
    # skipping 1st hiddel layer (nomralization layer), as we have normalized the data
    
    # Convolution Layers
    model.add(Conv2D(24, (5, 5), strides=(2, 2), input_shape=(66, 200, 3), activation='elu')) 
    model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='elu')) 
    model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='elu')) 
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    model.add(Dropout(0.2)) # not in original model. added for more robustness
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    
    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dropout(0.2)) # not in original model. added for more robustness
    model.add(Dense(256, activation='elu'))
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='relu'))
    model.add(Dense(10, activation='relu'))
    
    # output layer: turn angle (from 45-135, 90 is straight, <90 turn left, >90 turn right) # And speed as well 
    model.add(Dense(2)) 
    
    # since this is a regression problem not classification problem,
    # we use MSE (Mean Squared Error) as loss function
    optimizer = Adam(lr=1e-3) # lr is learning rate
    model.compile(loss='mean_squared_error', optimizer=optimizer)
    
    return model

model = nvidia_model()
print(model.summary())

Model: "Nvidia_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 31, 98, 24)        1824      
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 47, 36)        21636     
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 22, 48)         43248     
                                                                 
 conv2d_3 (Conv2D)           (None, 3, 20, 64)         27712     
                                                                 
 dropout (Dropout)           (None, 3, 20, 64)         0         
                                                                 
 conv2d_4 (Conv2D)           (None, 1, 18, 64)         36928     
                                                                 
 flatten (Flatten)           (None, 1152)             

# MobileNet (used for final car run)

In [16]:
#The dense(768) and dense(128) layers are the ones done on the heavier mobilenet model
model = Sequential()

#cmodel.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(224,224,3)))

tf.keras.layers.Resizing(

    224, 224, interpolation="bilinear", crop_to_aspect_ratio=False

)

pretrained_model = tf.keras.applications.mobilenet.MobileNet(include_top=False,weights="imagenet",input_shape=(224, 224, 3))

model.add(pretrained_model)

 

## Adding some more stuff

model.add(GlobalAveragePooling2D())

#model.add(Dense(768, activation ='relu',kernel_regularizer=l2(1e-6), bias_regularizer=l2(1e-6))) #1024

model.add(Dropout(0.2))

#model.add(Dense(128, activation ='relu',kernel_regularizer=l2(1e-7), bias_regularizer=l2(1e-7)))

#model.add(Dropout(0.2))

model.add(Dense(2))

 

model.compile(loss='mean_squared_error',optimizer='adam')

model.summary()

17225924/17225924 [==============================] - 2s 0us/step
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenet_1.00_224 (Functio  (None, 7, 7, 1024)       3228864   
 nal)                                                            
                                                                 
 global_average_pooling2d_1   (None, 1024)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_7 (Dropout)         (None, 1024)              0         
                                                                 
 dense_12 (Dense)            (None, 2)                 2050      
                                                                 
Total params: 3,230,914
Trainable params: 3,209,026
Non-trainable params: 21,888
________________________________________

### Custom early stopping (made for inceptionresnetv2 especially for kaggle contest as by hit and trial we noticed at a certain point validation loss barely changes but the training loss significantly decreases overtraining the model. So we kept checking early stopping for validation loss while making sure the difference between training and validation loss doesnt become huge.)

In [7]:
class CustomEarlyStopping(keras.callbacks.Callback):
    cont = 0
    val_loss = {}
    loss = {}
    best_weights = None
    def __init__(self, patience=None):
        super(CustomEarlyStopping, self).__init__()
        self.patience = 30
    
    def on_epoch_end(self, epoch, logs=None):
        #epoch += 1
        self.loss[epoch] = logs['loss']
        self.val_loss[epoch] = logs['val_loss']
        
        if self.val_loss[epoch]-self.loss[epoch]<=0.003:
            if self.val_loss[epoch] <= self.val_loss[self.cont] and epoch-self.cont <= self.patience:
                self.best_weights = self.model.get_weights()
                self.cont = epoch
            elif epoch-self.cont > self.patience:
                self.model.stop_training = True
                self.model.set_weights(self.best_weights)
        else:
            self.model.stop_training = True
            self.model.set_weights(self.best_weights)

            
hist = model.fit(
    train_images,
    validation_data=val_images,
    epochs=500,
    callbacks=[CustomEarlyStopping(patience=30)]
)

Epoch 1/500
65/65 [==============================] - 209s 3s/step - loss: 0.4616 - val_loss: 0.0612
Epoch 2/500
65/65 [==============================] - 79s 1s/step - loss: 0.0384 - val_loss: 0.0310
Epoch 3/500
65/65 [==============================] - 79s 1s/step - loss: 0.0309 - val_loss: 0.0227
Epoch 4/500
65/65 [==============================] - 79s 1s/step - loss: 0.0252 - val_loss: 0.0155
Epoch 5/500
65/65 [==============================] - 77s 1s/step - loss: 0.0189 - val_loss: 0.0121
Epoch 6/500
65/65 [==============================] - 78s 1s/step - loss: 0.0173 - val_loss: 0.0113
Epoch 7/500
65/65 [==============================] - 77s 1s/step - loss: 0.0160 - val_loss: 0.0115
Epoch 8/500
65/65 [==============================] - 78s 1s/step - loss: 0.0141 - val_loss: 0.0124
Epoch 9/500
65/65 [==============================] - 78s 1s/step - loss: 0.0140 - val_loss: 0.0133
Epoch 10/500
65/65 [==============================] - 79s 1s/step - loss: 0.0131 - val_loss: 0.0132
Epoch 11

KeyboardInterrupt: 

## Regular early stopping just monitoring validation loss

In [116]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True)
            

hist = model.fit(
    train_images,
    validation_data=val_images,
    epochs=500,
    callbacks=callback
)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [9]:
model.save('/kaggle/working')

In [10]:
!zip -r nvidia.zip /kaggle/working

  adding: kaggle/working/ (stored 0%)
  adding: kaggle/working/saved_model.pb (deflated 89%)
  adding: kaggle/working/assets/ (stored 0%)
  adding: kaggle/working/.virtual_documents/ (stored 0%)
  adding: kaggle/working/fingerprint.pb (stored 0%)
  adding: kaggle/working/keras_metadata.pb (deflated 93%)
  adding: kaggle/working/__notebook_source__.ipynb (deflated 44%)
  adding: kaggle/working/variables/ (stored 0%)
  adding: kaggle/working/variables/variables.data-00000-of-00001 (deflated 8%)
  adding: kaggle/working/variables/variables.index (deflated 66%)


In [ ]:
def plot_loss(result):
    loss = hist.history['loss']
    val_loss = hist.history['val_loss']
    plt.figure(figsize=(25, 8))
    plt.subplot(122)
    plt.plot(range(0,len(loss)), loss[0:], label='Train_loss')
    plt.plot(range(0,len(loss)), val_loss[0:], label='Test_loss')
    plt.title('Loss over ' + str(len(loss)) + ' Epochs', size=15)
    plt.legend()
    plt.grid(True)
    plt.show()
    
plot_loss(hist)

## Predicting on test set and saving csv file

In [12]:
def prediction(model,img_id):
    Y_pred = model.predict(test_images)
    Y_pred[:,1] = (np.rint(Y_pred[:,1])).astype(int)
    df = pd.DataFrame(Y_pred, columns = ['angle','speed'])
    df.insert(0,'image_id',img_id)
    df.to_csv('/kaggle/working/inceptionresnet.csv',index=False)
    return Y_pred

Y_pred = prediction(model,test_img_id)

7/7 [==============================] - 10s 2s/step


In [13]:
Y_pred

array([[0.5933392 , 0.        ],
       [0.72331357, 1.        ],
       [0.34900966, 1.        ],
       ...,
       [0.44728503, 1.        ],
       [0.35326004, 0.        ],
       [0.5227783 , 1.        ]], dtype=float32)

In [16]:
Y_pred_set = set(list(Y_pred[:,1]))
Y_pred_set

AttributeError: 'set' object has no attribute 'min'

In [22]:
print(Y_pred[:,0].max(), Y_pred[:,0].min())

1.0834672